## Fix stomach samples download

In [1]:
import os,sys
import pandas as pd
import numpy as np
import json
import cellxgene_census
import scanpy as sc
from sc_target_evidence_utils import cxg_utils, plotting_utils, preprocessing_utils, cellontology_utils

In [2]:
## Read cxg cleaned metadata
output_dir = '/nfs/team205/ed6/bin/sc_target_evidence/data/'
cxg_metadata = pd.read_csv(output_dir + 'cellxgene_hsapiens_donor_metadata.disease_relevant_annotation.csv', index_col=0)
stomach_datasets = cxg_metadata[cxg_metadata['tissue_general'] == 'stomach'].dataset_id.unique().tolist()

In [18]:
## Download datasets
for dataset_id in stomach_datasets:
    if not os.path.exists(f'{output_dir}/{dataset_id}.h5ad'):
        cellxgene_census.download_source_h5ad(
            dataset_id, 
            to_path=f'{output_dir}/{dataset_id}.h5ad', 
            census_version = "2023-07-25")

In [32]:
def process_adata(hlca_adata_full, disease_ids):
    OBS_COLS = [
            "assay", 
            "tissue_general", 
            "suspension_type", 
            "disease", 
            'donor_id', 
            'cell_type_ontology_term_id',
            'cell_type',
            'is_primary_data'
    ]
    
    STOMACH_TISSUES = ['body of stomach', 'cardia of stomach', 'stomach']
    
    hlca_adata = hlca_adata_full[(hlca_adata_full.obs['tissue'].isin(STOMACH_TISSUES)) & 
                            (hlca_adata_full.obs['disease_ontology_term_id'].isin(disease_ids)) &
                            (hlca_adata_full.obs['is_primary_data'])
                           ]

    hlca_adata_X = hlca_adata.raw.to_adata()
    raw_sum = np.array(hlca_adata_X.X.sum(1)).flatten()

#     hlca_adata_X = hlca_adata_X[:, hlca_adata_X.var_names.isin(all_genes_ls)].copy()
    hlca_adata_X.obs['tissue_general'] = 'stomach'
    hlca_adata_X.obs = hlca_adata_X.obs[OBS_COLS].copy()
    hlca_adata_X.obs['total_counts'] = raw_sum

    # Exclude blacklisted assays
    assay_blacklist = [
        'BD Rhapsody Targeted mRNA',
        'STRT-seq',
        'inDrop'
        ]

    hlca_adata_X = hlca_adata_X[~hlca_adata_X.obs['assay'].isin(assay_blacklist)].copy()
    return(hlca_adata_X)

In [3]:
stomach_adatas = [sc.read_h5ad(f'{output_dir}/{x}.h5ad') for x in stomach_datasets]
stomach_disease_ids = cxg_metadata[cxg_metadata['tissue_general'] == 'stomach'].disease_ontology_id.unique().tolist()

In [33]:
stomach_adatas = [process_adata(a, stomach_disease_ids) for a in stomach_adatas]

In [34]:
for i,a in enumerate(stomach_adatas):
    del stomach_adatas[i].obsm

In [35]:
import anndata as ad
adata = ad.concat(stomach_adatas, merge='same')

In [36]:
adata.var['feature_id'] = adata.var_names.values

In [37]:
del stomach_adatas
import gc
gc.collect()

2343

In [39]:
adata.obs['disease'].value_counts()

gastric cancer      116329
normal               82109
stomach disorder     27462
gastritis            26639
Name: disease, dtype: int64

In [40]:
## Load target data
targets = pd.read_csv(output_dir + 'TargetDiseasePairs_OpenTargets_cellXgeneID_12072023.csv', index_col=0)

In [41]:
disease_info_df = cxg_metadata[['disease_relevant_tissue', 'disease_ontology_id', 'disease']].drop_duplicates()
disease_name_mapper = dict(zip(disease_info_df['disease_ontology_id'], disease_info_df['disease']))
disease_tissue_mapper = dict(zip(disease_info_df['disease_ontology_id'], disease_info_df['disease_relevant_tissue']))

In [49]:
def clean_disease(pbulk_adata):
    '''Uniform disease naming.'''        
    DISEASE_RENAME = {'cardiomyopathy':[
    'arrhythmogenic right ventricular cardiomyopathy',
     'dilated cardiomyopathy',
     'non-compaction cardiomyopathy'],
  'renal cell carcinoma':['chromophobe renal cell carcinoma', 'clear cell renal carcinoma', 'nonpapillary renal cell carcinoma',
                         'kidney oncocytoma', ],
  'colorectal cancer': ['colorectal cancer', 'colorectal neoplasm'],
    'non-small cell lung carcinoma':['lung large cell carcinoma', 'non-small cell lung carcinoma']
    }

    disease_rename_rev = {x:k for k,v in DISEASE_RENAME.items() for x in v }

    pbulk_adata.obs['disease_name_original'] = pbulk_adata.obs['disease'].copy()
    pbulk_adata.obs['disease'] = [disease_rename_rev[x] if x in disease_rename_rev.keys() else x for x in pbulk_adata.obs.disease]


for disease_ontology_id in stomach_disease_ids:
    print(f"Running {disease_ontology_id}")
    ## Get list of all possible genes you might need 
    if "_" in disease_ontology_id:
        disease_ontology_id = disease_ontology_id.replace("_", ":")
    target_genes = targets[targets['cxg_matched_id'] == disease_ontology_id.replace(":", "_")].targetId.tolist()

    json_file =   output_dir + 'target_universe_dict.json'
    with open(json_file, "r") as json_file:
        universe_dict = json.load(json_file)

    all_genes_ls = sum([x for x in universe_dict.values()], [])
    all_genes_ls.extend(target_genes)
    all_genes_ls = list(set(all_genes_ls))

    ## Filter full dataset
    disease_ids = ['normal', disease_name_mapper[disease_ontology_id]]
    print(disease_ids)
    hlca_adata = adata[(adata.obs['disease'].isin(disease_ids))]

    hlca_adata = hlca_adata[:, hlca_adata.var_names.isin(all_genes_ls)].copy()
    hlca_adata.obs['disease_ontology_id'] = disease_ontology_id
    print(hlca_adata.obs['disease'].unique())
    
    ## Clean cell ontologies
    print("Cleaning cell ontologies...")
    graph = cellontology_utils.get_cellontology_graph(output_dir)

    # Exclude cell types with less than 5 cells
    ct_counts = hlca_adata.obs["cell_type_ontology_term_id"].value_counts()
    ontology_terms = ct_counts[ct_counts > 5].index.tolist()
    ct_rename_dict = cellontology_utils.rename_cts_to_high_level(ontology_terms, graph)
    hlca_adata.obs["high_level_cell_type_ontology_term_id"] = [
        ct_rename_dict[x] if x in ct_rename_dict.keys() else 'low_quality_annotation' for x in hlca_adata.obs["cell_type_ontology_term_id"] 
        ]

    plotting_utils.plot_celltype_rename(hlca_adata.obs, disease_ontology_id, graph, savedir=output_dir + '/plots/')
    
    ## Pseudo-bulk by cell ontology and sample
    print("Pseudo-bulking...")
    pbulk_adata = preprocessing_utils.anndata2pseudobulk(hlca_adata, 
                                     group_by=['high_level_cell_type_ontology_term_id', 'assay', 'suspension_type', 'disease', 'donor_id'],
                                     min_ncells=5       
                                    )
    print(pbulk_adata.obs['disease'].unique())
    clean_disease(pbulk_adata)
    print(pbulk_adata.obs['disease'].unique())
    pbulk_adata = pbulk_adata[pbulk_adata.obs['high_level_cell_type_ontology_term_id'] != 'low_quality_annotation'].copy()
    print(pbulk_adata.obs['disease'].unique())
    pbulk_adata.obs['high_level_cell_type'] = [f'{cellontology_utils.ontology2name(x, graph)} ({x})' for x in pbulk_adata.obs['high_level_cell_type_ontology_term_id'].tolist()]
    pbulk_adata.obs['sample_id'] = ['-'.join(x[1:]) for x in pbulk_adata.obs_names.str.split("-")]
    pbulk_adata.obs['disease_ontology_id'] = disease_ontology_id

    ## Save 
    print("Saving...")
#     if not keep_all_genes:
#         pbulk_adata.write_h5ad(output_dir + f'cellxgene_targets_{disease_ontology_id.replace(":","_")}.pbulk_all_OT_targets.h5ad')
#     else:
    pbulk_adata.write_h5ad(output_dir + f'cellxgene_targets_{disease_ontology_id.replace(":","_")}.pbulk_all_genes.h5ad')


Running MONDO:0004298
['normal', 'stomach disorder']
['stomach disorder' 'normal']
Cleaning cell ontologies...
Pseudo-bulking...
['stomach disorder' 'normal']
['stomach disorder' 'normal']
['stomach disorder' 'normal']
Saving...
Running MONDO:0001056
['normal', 'gastric cancer']
['gastric cancer' 'normal']
Cleaning cell ontologies...
Pseudo-bulking...
['gastric cancer' 'normal']
['gastric cancer' 'normal']
['normal']
Saving...
Running PATO:0000461
['normal', 'normal']
['normal']
Cleaning cell ontologies...
Pseudo-bulking...
['normal']
['normal']
['normal']
Saving...
Running MONDO:0004966
['normal', 'gastritis']
['normal' 'gastritis']
Cleaning cell ontologies...
Pseudo-bulking...
['normal' 'gastritis']
['normal' 'gastritis']
['normal' 'gastritis']
Saving...


In [46]:
pbulk_adata.obs['disease'].unique()

['normal', 'gastritis']
Categories (2, object): ['gastritis', 'normal']

In [43]:
output_dir

'/nfs/team205/ed6/bin/sc_target_evidence/data/'